### Model creation

In [1]:
import tensorflow as tf
print(tf.__version__)

2.13.0


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
import numpy as np
import pandas as pd

In [24]:
extracted_features = pd.read_json('artifacts/preprocessed_data.json')

In [28]:
extracted_features

,feature,class
0,"[-217.355255127, 70.2233810425, -130.385269165...",dog_bark
1,"[-424.0981750488, 109.3407669067, -52.91952514...",children_playing
2,"[-458.7911376953, 121.3841934204, -46.52066040...",children_playing
3,"[-413.8998413086, 101.6637268066, -35.42945098...",children_playing
4,"[-446.603515625, 113.6854095459, -52.402214050...",children_playing
...,...,...
8727,"[-398.5845031738, 135.5349578857, -50.72501754...",car_horn
8728,"[-346.4742126465, 86.3481521606, -45.168571472...",car_horn
8729,"[-303.8882446289, 111.3594512939, -45.94156646...",car_horn
8730,"[-344.1100769043, 125.4502105713, -54.90344238...",car_horn


In [29]:
### Split the dataset into independent and dependent dataset
X=np.array(extracted_features['feature'].tolist())
y=np.array(extracted_features['class'].tolist())

In [30]:
X.shape

(8732, 40)

In [34]:
### Label Encoding
y=np.array(pd.get_dummies(y))

In [35]:
y.shape


(8732, 10)

In [36]:
### Train Test Split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [37]:
X_train.shape

(6985, 40)

In [38]:
X_test.shape

(1747, 40)

In [39]:
y_train.shape

(6985, 10)

In [40]:
y_test.shape

(1747, 10)

In [41]:
### No of classes
num_labels=y.shape[1]

In [43]:
# Dense()

In [44]:
model=Sequential()
###first layer
model.add(Dense(100,input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###second layer
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###third layer
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))

###final layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [45]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               4100      
                                                                 
 activation (Activation)     (None, 100)               0         
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 200)               20200     
                                                                 
 activation_1 (Activation)   (None, 200)               0         
                                                                 
 dropout_1 (Dropout)         (None, 200)               0         
                                                                 
 dense_2 (Dense)             (None, 100)               2

In [46]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [47]:
## Trianing my model
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime 

num_epochs = 100
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='artifacts/audio_classification.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/100
211/219 [===========================>..] - ETA: 0s - loss: 10.6886 - accuracy: 0.1317
Epoch 1: val_loss improved from inf to 2.28517, saving model to artifacts\audio_classification.hdf5
219/219 [==============================] - 3s 7ms/step - loss: 10.4287 - accuracy: 0.1316 - val_loss: 2.2852 - val_accuracy: 0.1311
Epoch 2/100
 13/219 [>.............................] - ETA: 0s - loss: 2.8956 - accuracy: 0.1562

c:\Users\yashr\anaconda3\envs\audioclf\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


212/219 [============================>.] - ETA: 0s - loss: 2.5520 - accuracy: 0.1269
Epoch 2: val_loss improved from 2.28517 to 2.28114, saving model to artifacts\audio_classification.hdf5
219/219 [==============================] - 1s 5ms/step - loss: 2.5476 - accuracy: 0.1267 - val_loss: 2.2811 - val_accuracy: 0.1156
Epoch 3/100
207/219 [===========================>..] - ETA: 0s - loss: 2.3551 - accuracy: 0.1184
Epoch 3: val_loss improved from 2.28114 to 2.27437, saving model to artifacts\audio_classification.hdf5
219/219 [==============================] - 1s 5ms/step - loss: 2.3531 - accuracy: 0.1188 - val_loss: 2.2744 - val_accuracy: 0.1231
Epoch 4/100
216/219 [============================>.] - ETA: 0s - loss: 2.2613 - accuracy: 0.1444
Epoch 4: val_loss improved from 2.27437 to 2.24041, saving model to artifacts\audio_classification.hdf5
219/219 [==============================] - 1s 5ms/step - loss: 2.2615 - accuracy: 0.1442 - val_loss: 2.2404 - val_accuracy: 0.1649
Epoch 5/100
216/

In [48]:
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(test_accuracy[1])

0.7504293322563171


In [50]:
X_test[1]

array([-466.17956543,    1.09502459,  -34.01388931,   35.33934784,
        -14.88148022,  -19.12842941,   -0.58168399,  -16.13057899,
        -21.33907509,    7.67363453,  -29.16448975,  -18.95025253,
         -2.95799923,   -8.16232967,  -15.15310097,   -6.60480547,
         -7.56859732,    9.34064579,   14.43309975,   21.93418121,
         20.86139679,    1.33401227,  -19.22880363,   -4.6302309 ,
         -1.05647457,    3.21526694,   -6.98428106,  -16.41457748,
        -10.02864552,   13.00995445,    0.5334608 ,  -23.84339142,
        -15.26732063,    9.24573421,   10.36762619,   -0.58320105,
         -1.26240528,   17.70001602,   13.8474617 ,   -5.18628263])

In [51]:
model.predict_classes(X_test)

AttributeError: 'Sequential' object has no attribute 'predict_classes'

In [52]:
predictions = model.predict(X_test)

55/55 [==============================] - 0s 4ms/step


In [53]:
predicted_classes = predictions.argmax(axis=-1)

In [54]:
predicted_classes

array([5, 3, 4, ..., 1, 0, 2], dtype=int64)